# Práctica 6: *Fine-tuning en producción*

- Selecciona un modelo pre-entrenado como base y realiza *fine-tuning* para resolver alguna tarea de NLP que te parezca reelevante
  - Procura utilizar datasets pequeños para que sea viable
  - Recuerda las posibles tareas disponibles en HF `*For<task>`
- Desarrolla y pon en producción un prototipo del modelo
  - Incluye una URL pública donde podamos ver tu proyecto
  - Recomendamos usar framewoks de prototipado (*streamlit* o *gradio*) y el *free-tier* de *spaces* de hugging face
    - https://huggingface.co/spaces/launch
    - https://huggingface.co/docs/hub/spaces-sdks-streamlit
    - https://huggingface.co/docs/hub/spaces-sdks-gradio
- Reporta que tan bien se resolvió la tarea y que tan útil fue tu app
- Reporta retos y dificultades al realizar el *fine-tuning* y al poner tu modelo en producción

## Extra

- Utiliza [code carbon](https://codecarbon.io/#howitwork) para reportar las emisiones de tu app

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import evaluate
import numpy as np



In [2]:
# Dataset pequeño: clasificación de sentimientos (positivo / negativo)
dataset = load_dataset("glue", "sst2")

# Tokenizador y modelo base
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1}
)


c:\Users\Alejandro Torres\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Tokenización de cada oración
def tokenize(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Cargamos métrica estándar de SST-2 (accuracy)
metric = evaluate.load("glue", "sst2")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [5]:
training_args = TrainingArguments(
    output_dir="sentiment-model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    report_to="none"
)


In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(2000)),  # Subset de 2000 muestras
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


  0%|          | 0/750 [00:00<?, ?it/s]

c:\Users\Alejandro Torres\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/109 [00:00<?, ?it/s]

{'eval_loss': 0.3478076756000519, 'eval_accuracy': 0.8692660550458715, 'eval_runtime': 25.3099, 'eval_samples_per_second': 34.453, 'eval_steps_per_second': 4.307, 'epoch': 1.0}


c:\Users\Alejandro Torres\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

{'eval_loss': 0.4551349878311157, 'eval_accuracy': 0.8577981651376146, 'eval_runtime': 22.7915, 'eval_samples_per_second': 38.26, 'eval_steps_per_second': 4.782, 'epoch': 2.0}


c:\Users\Alejandro Torres\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/109 [00:00<?, ?it/s]

{'eval_loss': 0.5046898126602173, 'eval_accuracy': 0.8692660550458715, 'eval_runtime': 23.1763, 'eval_samples_per_second': 37.625, 'eval_steps_per_second': 4.703, 'epoch': 3.0}
{'train_runtime': 1136.5316, 'train_samples_per_second': 5.279, 'train_steps_per_second': 0.66, 'train_loss': 0.26966776021321615, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.26966776021321615, metrics={'train_runtime': 1136.5316, 'train_samples_per_second': 5.279, 'train_steps_per_second': 0.66, 'total_flos': 46272518193024.0, 'train_loss': 0.26966776021321615, 'epoch': 3.0})

In [9]:
from huggingface_hub import notebook_login
notebook_login()

In [10]:
model.push_to_hub("fine-tuned-sentiment-v1", commit_message="Agregué nombres legibles a las etiquetas")
tokenizer.push_to_hub("fine-tuned-sentiment-v1")


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Alejandro-03/fine-tuned-sentiment-v1/commit/3af51acda3fabb9f57c3f7407b5beb82d8305975', commit_message='Upload tokenizer', commit_description='', oid='3af51acda3fabb9f57c3f7407b5beb82d8305975', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Alejandro-03/fine-tuned-sentiment-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='Alejandro-03/fine-tuned-sentiment-v1'), pr_revision=None, pr_num=None)